# Chatbot zero

Neste exemplo implementamos um chatbot com descrição do perfil do agente, memória e acesso a informações. 

## Configuração

Carregue variáveis de ambiente

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

## Acesso ao LLM

Configure o LangChain para usar o LLM desejado.

OBS: Você também pode usar a função `langchain.chat_models.init_chat_model()`.

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2", 
    temperature=0.0, 
    max_tokens=1000,
)

## LLM sem memória

O LLM isolado não tem recursos de memória.

In [3]:
llm.invoke("Olá! Meu nome é José da Silva!").content

'Olá, José da Silva! Fico feliz em conhecê-lo. Como posso ajudá-lo hoje? Você tem alguma pergunta ou precisa de ajuda com algo em particular? Estou aqui para ajudar!'

In [4]:
llm.invoke("Qual é meu nome?").content

'Desculpe, mas não tenho acesso a informações pessoais sobre você. Posso ajudar com algo mais?'

## Chatbot com memória e conhecimento

Defina o template do prompt de sistema do chatbot

In [5]:
from datetime import datetime

prompt_template = """Você é uma assistente virtual amistosa e sempre disposta a ajudar.
Seu nome é Ana. Você atende clientes da empresa ACME.

Siga sempre as instruções abaixo:

- APENAS na sua primeira mensagem enviada ao cliente:
  fale o nome do cliente e deseje 'bom dia', 'boa tarde' ou 'boa noite', de acordo com o horário atual.
  Por exemplo: 'Bom dia, <nome do cliente>!'
- Dê respostas concisas.
- Utilize apenas as informações mencionadas na conversa para responder perguntas.

Informações gerais:

- Horário atual: {current_time}
- Data atual: {current_date}

Informações do cliente:

- Nome: José da Silva
- Localização atual: Campinas, São Paulo
- Comidas preferidas: vegetariana
"""

def make_system_prompt():
    current_time = datetime.now().strftime(r"%H:%M:%S")
    current_date = datetime.now().strftime(r"%A, %Y-%m-%d")
    system_prompt = prompt_template.format(current_time=current_time, current_date=current_date)
    return system_prompt

In [6]:
print(make_system_prompt())

Você é uma assistente virtual amistosa e sempre disposta a ajudar.
Seu nome é Ana. Você atende clientes da empresa ACME.

Siga sempre as instruções abaixo:

- APENAS na sua primeira mensagem enviada ao cliente:
  fale o nome do cliente e deseje 'bom dia', 'boa tarde' ou 'boa noite', de acordo com o horário atual.
  Por exemplo: 'Bom dia, <nome do cliente>!'
- Dê respostas concisas.
- Utilize apenas as informações mencionadas na conversa para responder perguntas.

Informações gerais:

- Horário atual: 00:49:54
- Data atual: Sunday, 2025-05-18

Informações do cliente:

- Nome: José da Silva
- Localização atual: Campinas, São Paulo
- Comidas preferidas: vegetariana



Defina o código do chatbot, incluindo as mensagens anteriores do diálogo (memória).

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

# Memória do chatbot
message_history = []

def chatbot(message: str):
    message_history.append(HumanMessage(message))
    prompt = [SystemMessage(make_system_prompt())] + message_history
    response = llm.invoke(prompt)
    message_history.append(response)
    print(response.content)


## Conversando com o chatbot

In [8]:
chatbot("Você pode recomendar um restaurante para mim?")

Bom dia, José da Silva!

Infelizmente, não tenho informações específicas sobre restaurantes na sua localização. Posso sugerir que você procure por opções de restaurante vegetariano em Campinas, São Paulo, e verifique as avaliações online para encontrar um lugar adequado para você.

Se precisar de mais ajuda, estou à disposição!


In [9]:
chatbot(
    "Hoje é meu aniversário e quero ir a um cinema também. "
    "Aliás, eu gosto muito de cinema e teatro. "
    "Mas olha, agora estou em São José dos Campos. "
)

Boa tarde, José da Silva!

Feliz aniversário! Lamento que você tenha mudado de cidade para o seu aniversário.

São José dos Campos tem várias opções de cinema e teatro. Você pode procurar por cinemas de luxo como o Cineplex ou o Cinemex, que oferecem uma experiência de cinema de alta qualidade. Além disso, há também teatros de arte e teatros de comédia na cidade.

Se precisar de mais recomendações ou ajuda para encontrar um local adequado, estou à disposição!


In [10]:
chatbot(
    "Bom, mas eu preciso sacar dinheiro. "
    "Eu tenho conta na Caixa. "
    "Tem alguma agência da Caixa aqui na cidade?"
)

Boa tarde, José da Silva!

Sim, há várias agências da Caixa em São José dos Campos. Você pode procurar por "Agência Caixa" ou "Caixa Econômica Federal" no seu aplicativo de navegação ou online para encontrar a agência mais próxima.

Se precisar de ajuda adicional, posso tentar ajudá-lo a encontrar a agência mais próxima!


In [11]:
chatbot("O que você sabe sobre mim?")

Bom dia, José da Silva!

Até o momento, sei que você é um cliente da ACME e que está atualmente em São José dos Campos, São Paulo. Além disso, você mencionou que gosta de cinema e teatro, e que tem uma conta na Caixa. Isso é tudo o que tenho sobre você até agora!


## Histórico da conversa

In [12]:
from langchain_core.messages import SystemMessage

def chat_history():
    output = ""
    for msg in message_history:
        output += msg.pretty_repr() + "\n\n"
    return output

print(chat_history())

================================ Human Message =================================

Você pode recomendar um restaurante para mim?

================================== Ai Message ==================================

Bom dia, José da Silva!

Infelizmente, não tenho informações específicas sobre restaurantes na sua localização. Posso sugerir que você procure por opções de restaurante vegetariano em Campinas, São Paulo, e verifique as avaliações online para encontrar um lugar adequado para você.

Se precisar de mais ajuda, estou à disposição!

================================ Human Message =================================

Hoje é meu aniversário e quero ir a um cinema também. Aliás, eu gosto muito de cinema e teatro. Mas olha, agora estou em São José dos Campos. 

================================== Ai Message ==================================

Boa tarde, José da Silva!

Feliz aniversário! Lamento que você tenha mudado de cidade para o seu aniversário.

São José dos Campos tem várias opções d